In [18]:
from xlwt import *
import os, re
import numpy as np
import pandas as pd

date = "20180717"
catch_month = int(date[4:6]) - 1 or 12
path = "/Users/zhewei/Desktop/KPI_result/"
file_list = [files for root, dirs, files in os.walk(path)][0]
head = ["Model Name", "Segment Name", "Delay Month", "Evaluation Date", "OP", "Last Month's Performance", "Health Target", "Q1",
        "Q2", "Q3", "Q4", "Year", "Performance Target", "Performance Target-5%", "M1", "M2", "M3", "M4", 
        "M5", "M6", "M7", "M8", "M9", "M10", "M11", "M12"]
report = dict(zip(head, [list() for i in range(len(head))]))
model_name = ["CAM", "ARM", "ATOM", "Tier1", "MM", "EXAM", "Reset", "EG", "Peeking", "VBASE"]
model_target = [80, 80, 80, 98,95, 95, 95, 95, 95, 95, 95]
model_name_target = dict(zip(model_name, model_target))
KPI_result = dict()
max_M = 0
max_file = None
latest_date = None
for file in file_list:
    M = int(file.split('.')[0][-2:])
    data = pd.read_excel(path + file)
    operate_file(M, data)
    if M > max_M:
        max_M = M
        max_file = path + file
        latest_date = list(data["Update date"])

report["Evaluation Date"] = latest_date
report["Last Month's Performance"] = report['M' + str(M)]
report["Segment Name"] = ["iACH & cELV" ,"ATOM Seg2", "CAM eligible", "SME", "CS", "YS", "SME", "CS", "YS", "ACH", "CC", "ATO"]

model_column = list(report["Model Name"])
for i in range(len(model_column)):
    for key, value in model_name_target.items():
        if key in model_column[i]:
            report["Health Target"].append("%.0f%%" % value) 

if 3 in KPI_result:
    report["Q1"] = ["%.0f%%" % (value*100) for value in (KPI_result[1] + KPI_result[2] + KPI_result[3])/3]
if 6 in KPI_result:
    report["Q2"] = ["%.0f%%" % (value*100) for value in (KPI_result[4] + KPI_result[5] + KPI_result[6])/3]
if 9 in KPI_result:
    report["Q3"] = ["%.0f%%" % (value*100) for value in (KPI_result[7] + KPI_result[8] + KPI_result[9])/3]
if 12 in KPI_result:
    report["Q4"] = ["%.0f%%" % (value*100) for value in (KPI_result[10] + KPI_result[11] + KPI_result[12])/3]
report["Year"] = ["%.0f%%" % (value*100) for value in np.array(list(KPI_result.values())).sum(axis = 0)/len(KPI_result.keys())]
# print(report)
for key, value in report.items():
    if len(value) == 0:
        report[key] = ["#"] * 12
report = np.array(pd.DataFrame(report, columns=head))
output_yearly(report, head, name="/Users/zhewei/Desktop/KPI_yearly" + date + ".xls")
# pd.DataFrame(report, columns=head)["Year"]

12


In [2]:
def operate_file(M, data):
    global report, KPI_result, model_name_target
    if not report["Model Name"]:
        model = [str(item) for item in list(data["Model"])]
        model[model.index("ATOM")] = "ATOM17V2"
        model[model.index("CAM17")] = "CAM17V2" 
        report["Model Name"] = model
    if not report["OP"]:
        report["OP"] = [str(item) for item in list(data["OP"])]
    if not report["Delay Month"]:
        report["Delay Month"] = [int(item) for item in list(data["Delay month"])]
    if not report["Performance Target"]:
        report["Performance Target"] = [str(item) for item in list(data["Target"])]
    if not report["Performance Target-5%"]:
        report["Performance Target-5%"] = [str(item) for item in list(data["target with 5% tolerance(absolutely)"])]
    report["M" + str(M)] = list(data["Catch rate"])
    KPI_result[M] = np.array((data["KPI with 5% tolerance(absolutely)"]))

In [16]:
import xlwt
from xlwt import *
import xlsxwriter

def output_yearly(report, head, name, date = "20180710"):
    """
    Output summarized excel yearly.
    
    @param report: a two-dims array generated from pandas.dataframe: report.
    @param head: a list generated from cloumns of report, a object of pandas dataframe.
    """
    row, column = report.shape
    model_list_reverse = list(report[:,0])[::-1]
    M_result = [i for i in range(len(head)) if len(head[i]) < 4 and head[i][0] == 'M'] 
    Q_result = [i for i in range(len(head)) if len(head[i]) == 2 and head[i][0] == 'Q'] 
    M_target = head.index("Performance Target-5%")
    Q_target = head.index("Health Target")
    Year = head.index("Year")
    LMP = head.index("Last Month's Performance")
    Eval_date = head.index("Evaluation Date")
    
    # set style 
    alignment = Alignment() # Create Alignment
    alignment.horz = xlwt.Alignment.HORZ_CENTER # May be: HORZ_GENERAL, HORZ_LEFT, HORZ_CENTER, HORZ_RIGHT, HORZ_FILLED, HORZ_JUSTIFIED, HORZ_CENTER_ACROSS_SEL, HORZ_DISTRIBUTED
    alignment.vert = xlwt.Alignment.VERT_CENTER # May be: VERT_TOP, VERT_CENTER, VERT_BOTTOM, VERT_JUSTIFIED, VERT_DISTRIBUTED
    font = Font() # 为样式创建字体
#     font.name = 'Times New Roman' 
    font.colour_index = 2
    font.underline = True  # 下划线
    pattern = Pattern()
    pattern.pattern = Pattern.SOLID_PATTERN  # 设置其模式为实型    
    pattern.pattern_fore_colour = 5
    
    pattern1 = Pattern()
    pattern1.pattern = Pattern.SOLID_PATTERN  # 设置其模式为实型    
    pattern1.pattern_fore_colour = 2
    # 0 = Black, 1 = White, 2 = Red, 3 = Green, 4 = Blue, 5 = Yellow, 6 = Magenta,
    # 7 = Cyan, 16 = Maroon, 17 = Dark Green, 18 = Dark Blue, 19 = Dark Yellow
    
    # different styles
    # normal style
    style = xlwt.XFStyle() # Create Style
    style.alignment = alignment
    # head style
    font1 = Font() 
    font1.bold = True
    font1.height = 200
    style_head = xlwt.XFStyle()
    style_head.font = font1
    style_head.alignment = alignment
    
    # Style1 
    style1 = XFStyle() # 初始化样式
    style1.font = font
    style1.alignment = alignment
    # Style2
    style2 = XFStyle()
    style2.alignment = alignment
    style2.pattern = pattern
    # Style3
    style3 = XFStyle()
    style3.alignment = alignment
    style3.pattern = pattern1
    
    workbook = xlwt.Workbook(encoding='utf-8')
#     workbook = xlsxwriter.Workbook("/Users/zhewei/Desktop/KPI_yearly" + date + ".xlsx")
    worksheet = workbook.add_sheet("sheet1")
#     worksheet = workbook.add_worksheet("sheet1")
    width = 256 * 18
    tall_style = xlwt.easyxf('font:height 360;') # 36pt,类型小初的字号
    print(row)
    for i in range(row + 1):
        worksheet.row(i).set_style(tall_style)
        merge_num = 1
        for j in range(column):
            if i == 0:
                temp_col = worksheet.col(j)
                temp_col.width = width   
                worksheet.write(i, j, head[j], style_head)
                continue
            if j == 0:
                merge_num = row + 1 - (model_list_reverse.index(report[i-1,j]) + i)
                try:
                    worksheet.write_merge(i, i + merge_num - 1, j, j, report[i-1][j], style)
                except:
                    continue
            elif j in M_result or j == LMP:
                curr_value = report[i-1][j]
                target_value = report[i-1][M_target]
                if curr_value !="#" and float(curr_value.strip("%")) < float(target_value.strip("%")):
                    worksheet.write(i, j, curr_value, style1)
                else:
                    worksheet.write(i, j, curr_value, style)
            elif j in Q_result or j == Year:
                try:
                    if report[i-1][j] == "#":
                        worksheet.write_merge(i, i + merge_num - 1, j, j, report[i-1][j], style)
                    else:
                        mean_value = sum([float(report[k][j].strip("%"))/100 for k in range(i-1, i-1 + merge_num)])/merge_num
                        mean_value = "%.0f%%" % (mean_value*100)
                        target_value = report[i-1][Q_target]
                        if float(mean_value.strip("%")) < float(target_value.strip("%")):
                            worksheet.write_merge(i, i + merge_num - 1, j, j, mean_value, style2)
                        else:
                            worksheet.write_merge(i, i + merge_num - 1, j, j, mean_value, style)
                except:
                     continue
            elif j == Eval_date:
                try:
                    if report[i-1][j] == report[i - 1 + merge_num - 1][j]:
                        m, d = int(report[i-1][j][5:7]), int(report[i-1][j][8:])
                        if (m == catch_month and d >= 15) or (m == (catch_month + 1)%12 and d <= 15):
                            worksheet.write_merge(i, i + merge_num - 1, j, j, report[i-1][j], style)
                        else:
                            worksheet.write_merge(i, i + merge_num - 1, j, j, report[i-1][j], style3)
                    else:
                        m, d = int(report[i-1][j][5:7]), int(report[i-1][j][8:])
#                         worksheet.write(i, j, report[i-1][j], style)
                        if (m == catch_month and d >= 15) or (m == (catch_month + 1)%12 and d <= 15):
                            worksheet.write(i, j, report[i-1][j], style)
                        else:
                            worksheet.write(i, j, report[i-1][j], style3)
                except:
                    continue 
            else:
                try:
                    if report[i-1][j] == report[i - 1 + merge_num - 1][j]:
                        worksheet.write_merge(i, i + merge_num - 1, j, j, report[i-1][j], style)
                    else:
                        worksheet.write(i, j, report[i-1][j], style)
                except:
                    continue
    peeking_dataframe = pd.read_csv("/Users/zhewei/Desktop/peeking_kpi.csv")
    peeking_data = dict(zip(list(peeking_dataframe["kpi_name"]), list(peeking_dataframe["kpi"])))
    worksheet.row(row + 1).set_style(tall_style)
    for j in range(column):
        if head[j] == "Model Name":
            worksheet.write(row + 1, j, "Peeking", style)
        elif head[j] == "Delay Month":
            worksheet.write(row + 1, j, 0, style)
        elif head[j] == "Evaluation Date":
            worksheet.write(row + 1, j, "2018-07-10", style)
        elif head[j] == "Last Month's Performance":
            value = list(filter(lambda x: len(x[0]) == 6, sorted(peeking_data.items(), key=lambda item: item[0])))[-1][1]
            if value == 1:
                worksheet.write(row + 1, j,  "100%", style)
            else:
                worksheet.write(row + 1, j,  "0", style1)
        elif head[j] == "Health Target":
            worksheet.write(row + 1, j,  "95%", style)
        elif head[j] == "Year" or ("Q" in head[j] and len(head[j]) < 4):
            if head[j] in peeking_data:
                value = peeking_data[head[j]]
                if value == 1:
                    worksheet.write(row + 1, j,  "100%", style)
                else:
                    worksheet.write(row + 1, j,  "0", style2)
            else:
                worksheet.write(row + 1, j, "#", style)
        elif len(head[j]) <= 3 and 'M' in head[j]:
            month = head[j][1:].rjust(2, '0')
            for key, value in peeking_data.items():
                if month == key[-2:]:
                    if peeking_data[key] == 1:
                        worksheet.write(row + 1, j, "100%", style)
                    else:
                        worksheet.write(row + 1, j, "0", style1)
                    break
            try:
                worksheet.write(row + 1, j, "#", style)
            except:
                continue
        else:
            worksheet.write(row + 1, j, "#", style)
    workbook.save(name + ".xls")

In [71]:
peeking_dataframe = pd.read_csv("/Users/zhewei/Desktop/peeking_kpi.csv")
peeking_data = dict(zip(list(peeking_dataframe["kpi_name"]), list(peeking_dataframe["kpi"])))

1